互联网中充斥着大量钓鱼欺诈类网站。这类非法网站通常试图掩人耳目、充当正规网站，而实际上却是在窃取用户的身份、密码、交易等重要信息
机器学习在信息安全领域中的一个重要应用就是用来识别这些钓鱼网站。fraudulent.csv在data文件夹中。

fraudulent.csv文件中含有10,086条数据，每条数据含有18个特征以及1个标签。各个特征的含义如下：

    contain_IP：网址中是否包涵ip，比如http://121.99.3.123/fake.html 包含ip。1表示包含，0表示不包含；
    is_long：网址字符是否过长。1表示网址过长，0表示网址不长；
    is_tinyurl：网址是否是短网址。比如https://bit.ly/2kXX6jV 就是短网址。1表示是短网址，0表示不是；
    contain_at：网址是否包含“@”符号。1表示包含，0表示不包含；
    contain_double_slash：网址是否包含“//”符号，该符号用来表示网址跳转。1表示包含，0表示不包含；
    contain_dash：网址是否包含“-”符号，该符号经常帮助用来伪装真网站，比如www.my-taobao.com 。 1表示包含，0表示不包含；
    contain_subdomain：网址是否包含子域名，比如www.ecnu.edu.cn 就包含edu和cn子域名。1表示包含，0表示不包含；
    is_SSL：网址是否是https安全链接。1表示包含，0表示不包含；
    with_long_history：网址所属的主域名存在的时间。1表示长久，0表示不长久；
    contain_icon：网址网页是否有小图标。1表示包含，0表示不包含；
    contain_ext_domain：该网页是否加载其他域名下的附件或者网页。1表示包含，0表示不包含；
    contain_email_to：该网页是否包含发送邮件的组件。1表示包含，0表示不包含；
    allow_right_click：该网页是否允许用户进行右击操作。1表示允许，0表示不允许；
    contain_pop_up_windowL：该网页是否包含弹窗。1表示包含，0表示不包含；
    contain_Iframe：该网页是否包含Iframe（嵌套网页）。1表示包含，0表示不包含；
    has_DNSRecord：网址是否有DNS记录。1表示有，0表示无；
    traffic：该网站的流量大小。1表示大，0表示小；
    google_rank：该网址在google搜索中的排名。1表示高于同类网站的平均排名，0表示低于同类网站的平均排名；   
    y：表示网站是否是钓鱼欺诈网站，1表示是，0表示不是。
    原始数据中含有大量缺失值，请自行处理这些缺失值（可以剔除缺失值过多的列或者使用众数填充等方法）。

将原始数据分为训练集、测试集（随机种子请设置为1）（若有需要可以将训练集进一步分为训练集和验证集）。

现在请建立一个二分类模型，使用训练集训练模型，再使用测试集测试模型。

评估指标为F1值

分类模型可采用：k-近邻、决策树、逻辑回归、支持向量机等。

可以与周围同学比较一下F1值的大小（越接近1越好），看看谁的数据预处理和分类模型更强。

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# 1. 读取数据
file_path = "/Users/yingnizhen/Desktop/fraudulent.csv"
data = pd.read_csv(file_path)

# 2. 查看数据基本信息
print("数据基本信息：")
print(data.info())

# 3. 数据预处理：处理缺失值
# 剔除缺失值过多的列（阈值设为50%）
missing_threshold = 0.5
data = data.dropna(axis=1, thresh=len(data) * (1 - missing_threshold))

# 填充剩余缺失值（使用众数填充）
imputer = SimpleImputer(strategy='most_frequent')
data_filled = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# 4. 特征和标签分离
X = data_filled.drop(columns=['y'])
y = data_filled['y']

# 5. 数据分割（训练集、测试集）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# 6. 模型训练与评估函数
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    print(f"{model.__class__.__name__} 的 F1 值: {f1:.4f}")
    return f1

# 7. 选择分类模型并评估
models = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(random_state=1),
    LogisticRegression(random_state=1, max_iter=1000),
    SVC(random_state=1)
]

best_model = None
best_f1 = 0

for model in models:
    f1 = evaluate_model(model, X_train, X_test, y_train, y_test)
    if f1 > best_f1:
        best_f1 = f1
        best_model = model

print(f"\n最佳模型: {best_model.__class__.__name__}，最佳 F1 值: {best_f1:.4f}")

数据基本信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10086 entries, 0 to 10085
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   contain_IP             9996 non-null   float64
 1   is_long                9997 non-null   float64
 2   is_tinyurl             9998 non-null   float64
 3   contain_at             10004 non-null  float64
 4   contain_double_slash   9970 non-null   float64
 5   contain_dash           9992 non-null   float64
 6   contain_subdomain      9989 non-null   float64
 7   is_SSL                 9990 non-null   float64
 8   with_long_history      7291 non-null   float64
 9   contain_icon           8728 non-null   float64
 10  contain_ext_domain     8559 non-null   float64
 11  contain_email_to       8007 non-null   float64
 12  allow_right_click      6679 non-null   float64
 13  contain_pop_up_window  9807 non-null   float64
 14  contain_Iframe         9427 non-null   float64

/Users/yingnizhen/Desktop/ynz-repository/ynz-repository-1/.venv/lib/python3.9/site-packages/joblib/externals/loky/backend/context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[Errno 2] No such file or directory: 'sysctl'
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "/Users/yingnizhen/Desktop/ynz-repository/ynz-repository-1/.venv/lib/python3.9/site-packages/joblib/externals/loky/backend/context.py", line 270, in _count_physical_cores
    cpu_info = subprocess.run(
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/subprocess.py", line 505, in run
    with Popen(*popenargs, **kwargs) as process:
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/subprocess.py", line 951, in __init__
    self._execute_child(a

SVC 的 F1 值: 0.8641

最佳模型: DecisionTreeClassifier，最佳 F1 值: 0.8732
